In [1]:
# Data processing
import pandas as pd
import numpy as np
import scipy.stats

# Visualization
import seaborn as sns

# Similarity
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('../data/songsDataset.csv')

In [3]:
df.head()

,'userID','songID','rating'
0,0,7171,5
1,0,8637,4
2,0,21966,4
3,0,35821,5
4,0,82446,5


In [4]:
# Remove extra quotes from column names
df.columns = df.columns.str.strip("'")

# Now the column names should be corrected
print(df.columns)

Index(['userID', 'songID', 'rating'], dtype='object')


In [5]:
# there is too much data at the moment, we must sample it
print(df.shape)
print(df['userID'].nunique())
print(df['songID'].nunique())

(2000000, 3)
200000
127771


In [6]:
# for ease of calculation, we are getting the songs with over 1000 ratings and also aggregating by songID
agg_ratings = df.groupby('songID').agg(number_of_ratings = ('rating', 'count')).reset_index()

agg_ratings_GT1000 = agg_ratings[agg_ratings['number_of_ratings']>1000]
agg_ratings_GT1000.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56 entries, 2118 to 125910
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   songID             56 non-null     int64
 1   number_of_ratings  56 non-null     int64
dtypes: int64(2)
memory usage: 1.3 KB


In [7]:
# merge the two dataframes (to ensure that we only have songs with over 100 ratings)
df_GT1000 = pd.merge(df, agg_ratings_GT1000[['songID']], on='songID', how='inner')
df_GT1000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72046 entries, 0 to 72045
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   userID  72046 non-null  int64
 1   songID  72046 non-null  int64
 2   rating  72046 non-null  int64
dtypes: int64(3)
memory usage: 1.6 MB


In [8]:
df_GT1000.head() 

,userID,songID,rating
0,0,90409,5
1,161,90409,5
2,466,90409,3
3,587,90409,5
4,695,90409,5


In [9]:
# train test split to be done here first

### Create the matrix

In [10]:
df_GT1000['rating'].value_counts()

rating
5    26664
1    13634
4    13429
3    11277
2     7042
Name: count, dtype: int64

In [11]:
matrix = df_GT1000.pivot_table(index='userID', columns='songID', values='rating')
matrix.head()

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
userID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# dataset normalization
normalized_matrix = matrix
row_means = normalized_matrix.apply(lambda row: row.mean(), axis=1)
normalized_matrix = normalized_matrix.sub(row_means, axis=0)
normalized_matrix = matrix.fillna(0)
print(normalized_matrix)

songID  2263    2726    3785    8063    12709   13859   16548   17029   \
userID                                                                   
0          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5          0.0     0.0     0.0     2.0     0.0     0.0     0.0     0.0   
7          0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
14         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...        ...     ...     ...     ...     ...     ...     ...     ...   
199976     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
199980     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
199988     0.0     5.0     0.0     0.0     0.0     0.0     0.0     0.0   
199990     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
199996     5.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

songID  19299   19670   ...  113954  

In [13]:
try:
    # Define a picked userID
    picked_userID = 4

    # Extract the first row vector from the normalized_matrix
    picked_row_vector = normalized_matrix.iloc[picked_userID].values.reshape(1, -1)

    # Extract the rest of the row vectors excluding the picked_userID
    rest_of_row_vectors = normalized_matrix.drop(picked_userID)

    # Compute cosine similarity between the first row vector and the rest of the row vectors
    similarity_scores = cosine_similarity(picked_row_vector, rest_of_row_vectors)

    # Print the similarity scores
    print(similarity_scores)

except KeyError as e:
    print(f"The picked_userID {picked_userID} is not found in the index.")
    print("Available indices to choose from:")
    print(normalized_matrix.index)

[[0. 0. 0. ... 0. 0. 0.]]


In [14]:
# for reference
normalized_matrix.index

Index([     0,      4,      5,      7,     14,     20,     31,     33,     40,
           46,
       ...
       199956, 199969, 199973, 199974, 199975, 199976, 199980, 199988, 199990,
       199996],
      dtype='int64', name='userID', length=53963)

In [15]:
# testing out with a particular userID
import pandas as pd

# Convert similarity_scores to a pandas Series
similarity_scores_series = pd.Series(similarity_scores.flatten(), index=rest_of_row_vectors.index)

# Find the index label of the maximum similarity score
most_similar_userID = similarity_scores_series.idxmax()

print(f"The top most common userID to picked_userID {picked_userID} is: {most_similar_userID}")

The top most common userID to picked_userID 4 is: 14


Predicted ratings (formula)
$$r_{xi} = \frac{\sum_{y \in N} s_{xy} \cdot r_{yi}}{\sum_{y \in N} s_{xy}}$$

To obtain the predicted rating of user x on item i. We need to first get the top N users in terms of similarity with respect to item i. 

Suppose that the use users are y, then we take the rating as:
$$\frac{{\text{(similarity scores of $y$ and $x$)} \cdot \text{(rating given on item $i$ on $y$)}}}{{\text{sum of similarity scores of $y$ and $x$}}}
$$

In [38]:
#================================================================================================
# predict the rating of a song for a user
#================================================================================================

# suppose that we want to find the predicted rating of songID 2263 for picked_userID 4
# top N similar userIDs
N = 10
songID = 2263
picked_userID = 4

# modularizing similarity scores as a function
def similarity_scores(picked_userID, normalized_matrix):
    try:
        # Extract the first row vector from the normalized_matrix
        picked_row_vector = normalized_matrix.iloc[picked_userID].values.reshape(1, -1)

        # Extract the rest of the row vectors excluding the picked_userID
        rest_of_row_vectors = normalized_matrix.drop(picked_userID)

        # Compute cosine similarity between the first row vector and the rest of the row vectors
        similarity_scores = cosine_similarity(picked_row_vector, rest_of_row_vectors)

        # Print the similarity scores
        return(similarity_scores)

    except KeyError as e:
        print(f"The picked_userID {picked_userID} is not found in the index.")
        print("Available indices to choose from:")
        print(normalized_matrix.index)

try:
    similarity_scores_series = pd.Series(similarity_scores(picked_userID, normalized_matrix).flatten(), index=rest_of_row_vectors.index)
    top_n_similar_userIDs = similarity_scores_series.nlargest(N).index.tolist()
    
    # retaining only the top N similar userIDs
    similarity_scores_series = similarity_scores_series[top_n_similar_userIDs]
    
    denom = np.sum(similarity_scores_series)
    matrix_adj = matrix.fillna(0)
    numer = np.sum(similarity_scores_series * matrix_adj.loc[top_n_similar_userIDs, songID])
    
    predicted_rating = numer / denom
    print(f"The predicted rating of songID {songID} for picked_userID {picked_userID} is: {predicted_rating}")
    
except KeyError as e:
    print(f"The songID {songID} is not found in the columns; pick from the list of available songIDs below:")
    print(f"{normalized_matrix.columns}")

The predicted rating of songID 2263 for picked_userID 4 is: 0.0


In [39]:
print(top_n_similar_userIDs)
matrix_adj.loc[top_n_similar_userIDs, songID]

[14, 986, 1071, 1101, 1412, 1495, 1591, 1968, 2173, 2753]


userID
14      0.0
986     0.0
1071    0.0
1101    0.0
1412    0.0
1495    0.0
1591    0.0
1968    0.0
2173    0.0
2753    0.0
Name: 2263, dtype: float64

In [66]:
# attributions

# https://colab.research.google.com/drive/1cN44RlIEaB28FTD30qFiHkN3rqcDgcng?usp=sharing#scrollTo=1Zo4UEFWJVyt
# https://www.youtube.com/watch?v=h9gpufJFF-0&ab_channel=ArtificialIntelligence-AllinOne
# https://www.youtube.com/watch?v=6BTLobS7AU8&t=728s&ab_channel=ArtificialIntelligence-AllinOne
# ChatGPT and Copilot to format code